<a href="https://colab.research.google.com/github/saif619/MachineLearningPython/blob/master/StackOverflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://miro.medium.com/max/1059/1*ZAT-ZZQaX80h-Hl1eJr7bA.png)

In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
!wget https://www.dropbox.com/s/5721wcs2guuykzl/stacksample.zip?dl=0

--2019-11-13 05:26:08--  https://www.dropbox.com/s/5721wcs2guuykzl/stacksample.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/5721wcs2guuykzl/stacksample.zip [following]
--2019-11-13 05:26:08--  https://www.dropbox.com/s/raw/5721wcs2guuykzl/stacksample.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5589cf3077a29b427cb96288c1.dl.dropboxusercontent.com/cd/0/inline/AsSfzg_fpjCg3tctYhjCd7QI_KvezATCo9RmiZNuIoop8tv0cMHGOoOL8lSEV64yFHW2Fmqx8zLNWERRVgWDPm8kJJzGKl3p2TwuNnhN748V1g/file# [following]
--2019-11-13 05:26:08--  https://uc5589cf3077a29b427cb96288c1.dl.dropboxusercontent.com/cd/0/inline/AsSfzg_fpjCg3tctYhjCd7QI_KvezATCo9RmiZNuIoop8tv0cMHGOoOL8lSEV64yFHW2Fmqx8zLNWERRVgWDPm8kJJzGKl3p2TwuNnhN748V1g/file
Resol

In [0]:
#For writing weights to google drive
#!pip install pydrive
def Gdrive():
  from google.colab import drive
  drive.mount('/content/gdrive')
  from pydrive.auth import GoogleAuth
  from pydrive.drive import GoogleDrive
  from google.colab import auth
  from oauth2client.client import GoogleCredentials
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)

In [0]:
Gdrive()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
os.listdir()

['.config', 'gdrive', 'stacksample.zip?dl=0', 'adc.json', 'sample_data']

In [0]:
!unzip stacksample.zip?dl=0

Archive:  stacksample.zip?dl=0
  inflating: Answers.csv             
  inflating: Questions.csv           
  inflating: Tags.csv                


## Importing the required Libraries

In [0]:
import pandas as pd
import numpy as np
import pickle
import time
import re
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import ToktokTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import hamming_loss
from sklearn.cluster import KMeans
from sklearn.utils import class_weight
from scipy.sparse import hstack

import warnings
warnings.filterwarnings("ignore")



## Reading data

In [0]:
def Load_data():
    questions = pd.read_csv("Questions.csv", encoding='latin1')
    labels = pd.read_csv("Tags.csv")
    return questions,labels

In [0]:
df_questions, labels = Load_data()

In [0]:
# Displaying top 10 tags
x = list(labels['Tag'].value_counts().head(10))

In [0]:
# converting tags type to str from object
labels['Tag'] = labels['Tag'].astype(str)

## Prepdata function

In [0]:
def prep_data(labels, df_questions):
  grouped_tags = labels.groupby("Id")['Tag'].apply(lambda tags: ' '.join(tags))
  grouped_tags = grouped_tags.reset_index()
  df_questions.drop(columns=['OwnerUserId', 'CreationDate', 'ClosedDate'], inplace=True)
  df_questions = df_questions.merge(grouped_tags, on='Id')
  new_df = df_questions[df_questions['Score']>5]
  new_df.drop_duplicates(inplace = True)
  new_df.drop(columns=['Id', 'Score'], inplace=True)
  new_df['Tags'] = new_df['Tag'].apply(lambda x: x.split())
  all_tags = [item for sublist in new_df['Tags'].values for item in sublist]
  return new_df, all_tags

In [0]:
new_df, all_tags = prep_data(labels, df_questions)

## Reducing the problem to the most common tags in the dataset

In [0]:
flat_list = [item for sublist in new_df['Tags'].values for item in sublist]

keywords = nltk.FreqDist(flat_list)

keywords = nltk.FreqDist(keywords)

frequencies_words = keywords.most_common(100)
tags_features = [word[0] for word in frequencies_words]

In [0]:
def most_common(tags):
    tags_filtered = []
    for i in range(0, len(tags)):
        if tags[i] in tags_features:
            tags_filtered.append(tags[i])
    return tags_filtered

In [0]:
new_df['Tags'] = new_df['Tags'].apply(lambda x: most_common(x))
new_df['Tags'] = new_df['Tags'].apply(lambda x: x if len(x)>0 else None)

In [0]:
new_df.dropna(subset=['Tags'], inplace=True)

In [0]:
new_df.shape

(63167, 4)



## In the next two columns: Body and Title, I'll use lots of text processing:

    Removing html format
    Lowering text
    Transforming abbreviations
    Removing punctuation (but keeping words like c# since it's the most popular tag)
    Lemmatizing words
    Removing stop words

In [0]:
new_df['Body'] = new_df['Body'].apply(lambda x: BeautifulSoup(x).get_text())

In [0]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub(r"\'\n", " ", text)
    text = re.sub(r"\'\xa0", " ", text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [0]:
new_df['Body'] = new_df['Body'].apply(lambda x: clean_text(x))

In [0]:
token = ToktokTokenizer()

In [0]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [0]:
punct = '!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~'

In [0]:
def strip_list_noempty(mylist):
    newlist = (item.strip() if hasattr(item, 'strip') else item for item in mylist)
    return [item for item in newlist if item != '']

In [0]:
def clean_punct(text): 
    words=token.tokenize(text)
    punctuation_filtered = []
    regex = re.compile('[%s]' % re.escape(punct))
    remove_punctuation = str.maketrans(' ', ' ', punct)
    for w in words:
        if w in tags_features:
            punctuation_filtered.append(w)
        else:
            punctuation_filtered.append(regex.sub('', w))
  
    filtered_list = strip_list_noempty(punctuation_filtered)
        
    return ' '.join(map(str, filtered_list))

In [0]:
new_df['Body'] = new_df['Body'].apply(lambda x: clean_punct(x)) 

In [0]:
nltk.download('stopwords')
lemma=WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Lemmatization and Stopwords

In [0]:
def lemitizeWords(text):
    words=token.tokenize(text)
    listLemma=[]
    for w in words:
        x=lemma.lemmatize(w, pos="v")
        listLemma.append(x)
    return ' '.join(map(str, listLemma))

def stopWordsRemove(text):
    
    stop_words = set(stopwords.words("english"))
    
    words=token.tokenize(text)
    
    filtered = [w for w in words if not w in stop_words]
    
    return ' '.join(map(str, filtered))

In [0]:
nltk.download('wordnet')
new_df['Body'] = new_df['Body'].apply(lambda x: lemitizeWords(x)) 
new_df['Body'] = new_df['Body'].apply(lambda x: stopWordsRemove(x)) 

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


## DataPreparation for Title

In [0]:
new_df['Title'] = new_df['Title'].apply(lambda x: str(x))
new_df['Title'] = new_df['Title'].apply(lambda x: clean_text(x)) 
new_df['Title'] = new_df['Title'].apply(lambda x: clean_punct(x)) 
new_df['Title'] = new_df['Title'].apply(lambda x: lemitizeWords(x)) 
new_df['Title'] = new_df['Title'].apply(lambda x: stopWordsRemove(x)) 

In [0]:
new_df.head()

,Title,Body,Tag,Tags
1,good branch merge tutorials tortoisesvn,really good tutorials explain branch merge apa...,svn tortoisesvn branch branching-and-merging,[svn]
2,asp.net site map,anyone get experience create sql-based asp.net...,sql asp.net sitemap,"[sql, asp.net]"
3,function create color wheel,something pseudo-solved many time never quite ...,algorithm language-agnostic colors color-space,[algorithm]
4,add script functionality .net applications,little game write c use database back-end trad...,c# .net scripting compiler-construction,"[c#, .net]"
5,use nest class case,work collection class use video playback recor...,c++ oop class nested-class,"[c++, oop, class]"


## Data preparation for tags

In [0]:
X1 = new_df['Body']+' '+new_df['Title']
X2 = new_df['Title']
y = new_df['Tags']

In [0]:
X1[4]

'little game write c use database back-end trade card game want implement function card script mean essentially interface icard card class implement public class card056 icard contain function call game make thing maintainablemoddable would like class card source code database essentially compile first use addchange card add database tell application refresh without need assembly deployment especially since would talk 1 assembly per card mean hundreds assemblies possible register class source file instantiate etc icard card current new mygamecardlibrarycard056 card current onenterplay ref currentgamestate language c# extra bonus possible write script .net language add script functionality .net applications'

In [0]:
X2[4]

'add script functionality .net applications'

In [0]:
multilabel_binarizer = MultiLabelBinarizer()
y_bin = multilabel_binarizer.fit_transform(y)

In [0]:
X1.shape

(63167,)

## Creating Features from Text

In [0]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [0]:
maxlen = 180

In [0]:

max_words = 5000
tokenizer = Tokenizer(num_words=max_words, lower=True)
tokenizer.fit_on_texts(X1)
#tokenizer.fit_on_texts(df_questions.Title)
#tokenizer.fit_on_texts(df_questions.Body)

def get_features(text_series):
    """
    transforms text data to feature_vectors that can be used in the ml model.
    tokenizer must be available.
    """
    sequences = tokenizer.texts_to_sequences(text_series)
    return pad_sequences(sequences, maxlen=maxlen)


In [0]:
def prediction_to_label(prediction):
    tag_prob = [(tags_features[i], prob) for i, prob in enumerate(prediction.tolist())]
    return dict(sorted(tag_prob, key=lambda kv: kv[1], reverse=True))

In [0]:
x1 = get_features(X1)


In [0]:
x1.shape

(63167, 180)

In [0]:
y_bin.shape

(63167, 100)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(x1, y_bin, test_size = 0.1, 
                                                    random_state = 0) 


## Evaluation metrics Average Jacard loss

In [0]:
def avg_jacard(y_true,y_pred):
    '''
    see https://en.wikipedia.org/wiki/Multi-label_classification#Statistics_and_evaluation_metrics
    '''
    jacard = np.minimum(y_true,y_pred).sum(axis=1) / np.maximum(y_true,y_pred).sum(axis=1)
    
    return jacard.mean()*100

def print_score(y_pred, clf):
    print("Clf: ", clf.__class__.__name__)
    print("Jacard score: {}".format(avg_jacard(y_test, y_pred)))
    print("Hamming loss: {}".format(hamming_loss(y_pred, y_test)*100))
    print("---")    

In [0]:
len(flat_list), len(set(flat_list))

(224129, 14883)

## Calculating weights for labels

In [0]:
class_weight = class_weight.compute_class_weight('balanced', tags_features, [i for i in flat_list if i in tags_features])
class_weight_dict = dict(enumerate(class_weight))


## Keras Model

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, GlobalMaxPool1D, Dropout, Conv1D
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.losses import binary_crossentropy
from keras.optimizers import Adam
from keras.models import Model

filter_length = 300
max_len = 180

def RNN():
    inputs = Input(name='inputs', shape=[max_len])
    layer = Embedding(max_words, 30, input_length=max_len)(inputs)
    layer = LSTM(300)(layer)
    layer = Dense(500, name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(200, name='FC2')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(100, name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs, outputs=layer)
    return model
  
model = RNN()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#model.summary()







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
import time

filepath = '/content/gdrive/My Drive/model-stackoverflow.h5'
callbacks = [
    ReduceLROnPlateau(), 
    EarlyStopping(patience=4), 
    ModelCheckpoint(filepath, monitor='val_acc', verbose=1, 
                    save_weights_only=True, 
                    mode='auto', period=10, save_best_only=False)
            ]

history = model.fit(X_train, y_train,
                    class_weight=class_weight,
                    epochs=12,
                    batch_size=30,
                    validation_split=0.1,
                    callbacks=callbacks)




Train on 51165 samples, validate on 5685 samples
Epoch 1/12





51165/51165 [==============================] - 602s 12ms/step - loss: 0.0751 - acc: 0.9838 - val_loss: 0.0582 - val_acc: 0.9856
Epoch 2/12
51165/51165 [==============================] - 590s 12ms/step - loss: 0.0495 - acc: 0.9868 - val_loss: 0.0401 - val_acc: 0.9881
Epoch 3/12
51165/51165 [==============================] - 585s 11ms/step - loss: 0.0373 - acc: 0.9889 - val_loss: 0.0338 - val_acc: 0.9897
Epoch 4/12
51165/51165 [==============================] - 590s 12ms/step - loss: 0.0325 - acc: 0.9900 - val_loss: 0.0314 - val_acc: 0.9903
Epoch 5/12
51165/51165 [==============================] - 593s 12ms/step - loss: 0.0298 - acc: 0.9906 - val_loss: 0.0301 - val_acc: 0.9907
Epoch 6/12
51165/51165 [==============================] - 583s 11ms/step - loss: 0.0279 - acc: 0.9910 - val_loss: 0.0291 - val_acc: 0.9908
Epoch 7/12
51165/51165 [==============================] - 582s 11ms/step - loss: 0.0263 - acc: 0.9914 - val_l

In [0]:
model.save_weights("/content/gdrive/My Drive/model-stackoverflow13Nov.h5")

In [0]:
#model.load_weights('model-stackoverflow13Nov.h5')

In [0]:


#cnn_model = keras.models.load_model('model-stackoverflow.h5')
metrics = model.evaluate(X_test, y_test)
print("{}: {}".format(model.metrics_names[0], metrics[0]))
print("{}: {}".format(model.metrics_names[1], metrics[1]))

6317/6317 [==============================] - 22s 4ms/step
loss: 0.030487704828372104
acc: 0.9907266205391666


## Predicting on Test split

In [0]:
new_df.reset_index(inplace=True)

In [0]:
X_test.shape

(6317, 180)

In [0]:
s=101
z = X_test[s].reshape(1 ,180)

In [0]:
l = model.predict(z).reshape(100, )
p1 = prediction_to_label(l)
df = pd.DataFrame()
df['label'] = p1.keys()
df['probability'] = p1.values()

In [0]:
#print(y_test[s],tags_features[list(y_test[s]).index(1)])
c=[i for i,j in enumerate (y_test[s]) if j==1]
print('actual tags ', [tags_features[i] for i in c])
df.head(10)

actual tags  ['database', 'linq']


,label,probability
0,linq,0.784859
1,database,0.366803
2,c,0.322816
3,c#,0.092800
4,algorithm,0.030599
5,iphone,0.027242
6,mongodb,0.023946
7,ruby-on-rails,0.012027
8,android,0.010132
9,wpf,0.009588


In [0]:
np.where(df['label'] == 'cocoa-touch')  # to check position of tag if not present in top 10

(array([28]),)